In [1]:
import pyarrow.parquet as pq
import pandas as pd
import re
from IPython.display import display

dataset = pq.ParquetDataset('th_40_challenge/w32processes/')
table = dataset.read()
df = table.to_pandas()

#pd.set_option('display.max_colwidth', None)

print('Student ID: 82R4k2JOBFYpjzrPT6xpDrASy2C3')

Student ID: 82R4k2JOBFYpjzrPT6xpDrASy2C3


In [2]:
df_elevated = df[df['username'].str.contains('NT AUTHORITY')]
#display(df)

In [3]:
LEGIT_SHELL = "explorer.exe"

def suspicious_shell(df):
    mask_shell = df['name'].str.lower().str.contains("shell", na=False) | df['path'].str.lower().str.contains("winlogon", na=False)
    non_explorer = df[(mask_shell) & (~df['name'].str.lower().str.contains(LEGIT_SHELL))]
    return non_explorer

df2 = suspicious_shell(df_elevated)

print("Suspicious shells:", len(df2))
display(df2)

Suspicious shells: 23


,arguments,hostname,name,path,pid,username
80,"""C:\Windows\SystemApps\ShellExperienceHost_cw5...",177PAMWS,ShellExperienceHost.exe,C:\Windows\SystemApps\ShellExperienceHost_cw5n...,5868,NT AUTHORITY\SYSTEM
159,"""C:\Windows\SystemApps\ShellExperienceHost_cw5...",204OQRWS,ShellExperienceHost.exe,C:\Windows\SystemApps\ShellExperienceHost_cw5n...,380,NT AUTHORITY\SYSTEM
39,"""C:\Windows\SystemApps\ShellExperienceHost_cw5...",236SLDES,ShellExperienceHost.exe,C:\Windows\SystemApps\ShellExperienceHost_cw5n...,4880,NT AUTHORITY\SYSTEM
60,"""C:\Windows\SystemApps\ShellExperienceHost_cw5...",248PAMEE,ShellExperienceHost.exe,C:\Windows\SystemApps\ShellExperienceHost_cw5n...,6080,NT AUTHORITY\SYSTEM
97,"""C:\Windows\SystemApps\ShellExperienceHost_cw5...",339BMXEE,ShellExperienceHost.exe,C:\Windows\SystemApps\ShellExperienceHost_cw5n...,1940,NT AUTHORITY\SYSTEM
2,"""C:\Windows\SystemApps\ShellExperienceHost_cw5...",439JSUSS,ShellExperienceHost.exe,C:\Windows\SystemApps\ShellExperienceHost_cw5n...,5564,NT AUTHORITY\SYSTEM
3,"""C:\Windows\SystemApps\ShellExperienceHost_cw5...",460PAMEB,ShellExperienceHost.exe,C:\Windows\SystemApps\ShellExperienceHost_cw5n...,5356,NT AUTHORITY\SYSTEM
24,"""C:\Windows\SystemApps\ShellExperienceHost_cw5...",504PFWNN,ShellExperienceHost.exe,C:\Windows\SystemApps\ShellExperienceHost_cw5n...,6160,NT AUTHORITY\SYSTEM
25,"""C:\Windows\SystemApps\ShellExperienceHost_cw5...",549RIUET,ShellExperienceHost.exe,C:\Windows\SystemApps\ShellExperienceHost_cw5n...,2844,NT AUTHORITY\SYSTEM
37,C:\Windows\System32\WindowsPowerShell\v1.0\New...,549RIUET,Powershell.exe,C:\Windows\System32\WindowsPowerShell\v1.0\Pow...,6068,NT AUTHORITY\SYSTEM


In [4]:
CRITICAL_PROCESSES = [
    "explorer.exe", "lsass.exe", "svchost.exe", "services.exe",
    "winlogon.exe", "csrss.exe", "smss.exe", "dllhost.exe"
]

# Procesos que suelen usarse como "injectors"
INJECTION_TOOLS = [
    "rundll32.exe", "regsvr32.exe", "mshta.exe", "powershell.exe",
    "cmd.exe", "wmic.exe", "cscript.exe", "wscript.exe"
]

# Patrones sospechosos en argumentos
SUSPICIOUS_ARG_PATTERNS = [
    r"\.dll", r"\.ocx", r" -enc", r" -encodedcommand",
    r"shellcode", r"inject", r"CreateRemoteThread", r"VirtualAlloc"
]

def hunt_process_injection(df, threshold=5):
    """
    Busca indicios de process injection en procesos críticos o herramientas de inyección.
    """
    df = df.copy()
    
    mask_critical_with_args = (
        df['name'].str.lower().isin([p.lower() for p in CRITICAL_PROCESSES]) &
        df['arguments'].str.strip().ne("")
    )
    crit_susp = df[mask_critical_with_args]
    
    pattern = "|".join(SUSPICIOUS_ARG_PATTERNS)
    mask_injectors = (
        df['name'].str.lower().isin([p.lower() for p in INJECTION_TOOLS]) &
        df['arguments'].str.lower().str.contains(pattern, na=False)
    )
    injectors = df[mask_injectors]
    
    host_counts = df[df['name'].str.lower().isin([p.lower() for p in CRITICAL_PROCESSES])] \
                    .groupby('name')['hostname'].nunique()
    rare_procs = host_counts[host_counts < threshold].index.tolist()
    rare_crit = df[df['name'].str.lower().isin([p.lower() for p in rare_procs])]
    
    results = pd.concat([crit_susp, injectors, rare_crit]).drop_duplicates()
    return results

df_1 = hunt_process_injection(df_elevated)

display(df_1)

,arguments,hostname,name,path,pid,username
0,C:\Windows\system32\svchost.exe -k LocalServic...,168SKJWT,svchost.exe,C:\Windows\system32,3580,NT AUTHORITY\SYSTEM
1,C:\Windows\System32\svchost.exe -k LocalServic...,168SKJWT,svchost.exe,C:\Windows\System32,2392,NT AUTHORITY\SYSTEM
2,C:\Windows\system32\svchost.exe -k NetworkServ...,168SKJWT,svchost.exe,C:\Windows\system32,2780,NT AUTHORITY\SYSTEM
3,C:\Windows\system32\svchost.exe -k DcomLaunch,168SKJWT,svchost.exe,C:\Windows\system32,6220,NT AUTHORITY\SYSTEM
9,C:\Windows\system32\svchost.exe -k RPCSS,168SKJWT,svchost.exe,C:\Windows\system32,3960,NT AUTHORITY\SYSTEM
...,...,...,...,...,...,...
53,C:\Windows\system32\svchost.exe -k NetworkService,992BMXWC,svchost.exe,C:\Windows\system32,3724,NT AUTHORITY\SYSTEM
58,C:\Windows\System32\svchost.exe -k LocalServic...,992BMXWC,svchost.exe,C:\Windows\System32,688,NT AUTHORITY\SYSTEM
81,C:\Windows\Explorer.EXE,992BMXWC,Explorer.EXE,C:\Windows,2456,NT AUTHORITY\SYSTEM
4,C:\Windows\System32\regsvr32 /u /n /s /i:http:...,168SKJWT,regsvr32.exe,C:\Windows\System32\regsvr32.exe,3292,NT AUTHORITY\SYSTEM


In [6]:
n_elevated_processes = df[df['username'].str.contains('NT AUTHORITY')]
elevated_consoles = n_elevated_processes[n_elevated_processes['name'].str.contains('cmd')]
print('Nº of elevated cmds: ' + str(len(elevated_consoles)))

display(elevated_consoles)

Nº of elevated cmds: 8


,arguments,hostname,name,path,pid,username
72,C:\Windows\System32\cmd.exe /C C:\Windows\Syst...,236SLDES,cmd.exe,C:\Windows\System32\cmd.exe,1664,NT AUTHORITY\SYSTEM
144,C:\Windows\System32\cmd.exe,554PAMET,cmd.exe,C:\Windows\System32\cmd.exe,4752,NT AUTHORITY\SYSTEM
237,C:\Windows\System32\cmd.exe,554PAMET,cmd.exe,C:\Windows\System32\cmd.exe,72,NT AUTHORITY\SYSTEM
3,C:\Windows\System32\cmd.exe /C C:\Program File...,608KVKWB,cmd.exe,C:\Windows\System32\cmd.exe,6360,NT AUTHORITY\SYSTEM
33,C:\Windows\System32\cmd.exe /C C:\Users\darc.s...,608KVKWB,cmd.exe,C:\Windows\System32\cmd.exe,2112,NT AUTHORITY\SYSTEM
21,C:\Windows\System32\cmd.exe /C C:\Windows\Syst...,756OQREN,cmd.exe,C:\Windows\System32\cmd.exe,1896,NT AUTHORITY\SYSTEM
36,C:\Windows\System32\cmd.exe,829SKJST,cmd.exe,C:\Windows\System32\cmd.exe,1156,NT AUTHORITY\SYSTEM
93,C:\Windows\System32\cmd.exe /C C:\Windows\run_...,850AVYNC,cmd.exe,C:\Windows\System32\cmd.exe,4860,NT AUTHORITY\SYSTEM


In [10]:
host_count = df_elevated.groupby('name')['hostname'].nunique()

# Less than 10 hosts
rare_procs = host_count[host_count < 10].index.tolist()
rare_elevated = elevated[elevated['name'].isin(rare_procs)]

display(rare_elevated)

,arguments,hostname,name,path,pid,username
4,C:\Windows\System32\regsvr32 /u /n /s /i:http:...,168SKJWT,regsvr32.exe,C:\Windows\System32\regsvr32.exe,3292,NT AUTHORITY\SYSTEM
22,taskhostex.exe,168SKJWT,taskhostex.exe,C:\Windows\system32,1088,NT AUTHORITY\SYSTEM
29,C:\Windows\system32\wbem\wmiprvse.exe,168SKJWT,wmiprvse.exe,C:\Windows\system32\wbem,744,NT AUTHORITY\SYSTEM
49,C:\Windows\System32\mavinject.exe 3328 /INJECT...,168SKJWT,mavinject.exe,C:\Windows\System32\mavinject.exe,2300,NT AUTHORITY\SYSTEM
60,C:\Program Files\Common Files\Services\procdum...,168SKJWT,procdump.exe,C:\Program Files\Common Files\Services\,4664,NT AUTHORITY\SYSTEM
...,...,...,...,...,...,...
29,C:\Windows\System32\VBoxService.exe,992BMXWC,VBoxService.exe,C:\Windows\System32,3272,NT AUTHORITY\SYSTEM
30,C:\Windows\System32\WindowsPowerShell\v1.0\pow...,992BMXWC,Powershell.exe,C:\Windows\System32\WindowsPowerShell\v1.0\Pow...,1708,NT AUTHORITY\SYSTEM
61,C:\Windows\system32\sppsvc.exe,992BMXWC,sppsvc.exe,C:\Windows\system32,5188,NT AUTHORITY\SYSTEM
69,"""C:\Program Files\Windows Media Player\wmpnetw...",992BMXWC,wmpnetwk.exe,C:\Program Files\Windows Media Player,3960,NT AUTHORITY\SYSTEM


In [17]:
js_proc = df_elevated[df_elevated['arguments'].str.contains('.js')]
print('Nº of JavaScritp processes: '+ str(len(js_proc)))

js_proc = df_elevated[df_elevated['arguments'].str.contains('.js') & ((df_elevated['name'] != 'slack.exe') & (df_elevated['name'] != 'cscript.exe'))]
print('Nº of suspicious JavaScritp processes: '+ str(len(js_proc)))

mshta_proc = df_elevated[df_elevated['arguments'].str.contains('mshta')]
print('Nº of mshta processes: '+ str(len(mshta_proc)))

vbs_proc = df_elevated[df_elevated['arguments'].str.contains('vb')]
print('Nº of vbs processes: '+ str(len(vbs_proc)))

cscript_proc = df_elevated[df_elevated['arguments'].str.contains('cscript')]
print('Nº of cscript.exe processes: '+ str(len(cscript_proc)))

display(cscript_proc[['name','pid','arguments']])

Nº of JavaScritp processes: 0
Nº of suspicious JavaScritp processes: 0
Nº of mshta processes: 0
Nº of vbs processes: 0
Nº of cscript.exe processes: 0


,name,pid,arguments


In [19]:
known_services = [
    'services.exe', 'svchost.exe', 'lsass.exe', 'wininit.exe',
    'taskhostw.exe', 'spoolsv.exe', 'smss.exe', 'csrss.exe',
    'dllhost.exe', 'dcomlaunch.exe', 'winlogon.exe', 'wmiprvse.exe']

service_like = df[
    (df['name'].str.lower().isin(known_services))
]

service_like = service_like[
    service_like['arguments'].notna() & 
    (service_like['arguments'].str.strip() != '')
]

service_like_elevated = service_like[service_like['username'].str.contains('NT AUTHORITY')]

service_host_counts = service_like_elevated.groupby('name')['hostname'].nunique()

rare_services = service_host_counts[service_host_counts < 5].index.tolist()

susp_services = service_like_elevated[service_like_elevated['name'].isin(rare_services)]

display(service_like_elevated)
print(service_host_counts)

,arguments,hostname,name,path,pid,username
0,C:\Windows\system32\svchost.exe -k LocalServic...,168SKJWT,svchost.exe,C:\Windows\system32,3580,NT AUTHORITY\SYSTEM
1,C:\Windows\System32\svchost.exe -k LocalServic...,168SKJWT,svchost.exe,C:\Windows\System32,2392,NT AUTHORITY\SYSTEM
2,C:\Windows\system32\svchost.exe -k NetworkServ...,168SKJWT,svchost.exe,C:\Windows\system32,2780,NT AUTHORITY\SYSTEM
3,C:\Windows\system32\svchost.exe -k DcomLaunch,168SKJWT,svchost.exe,C:\Windows\system32,6220,NT AUTHORITY\SYSTEM
9,C:\Windows\system32\svchost.exe -k RPCSS,168SKJWT,svchost.exe,C:\Windows\system32,3960,NT AUTHORITY\SYSTEM
...,...,...,...,...,...,...
46,C:\Windows\system32\svchost.exe -k RPCSS,992BMXWC,svchost.exe,C:\Windows\system32,3808,NT AUTHORITY\SYSTEM
49,\SystemRoot\System32\smss.exe,992BMXWC,smss.exe,\SystemRoot\System32,6072,NT AUTHORITY\SYSTEM
52,C:\Windows\System32\svchost.exe -k LocalServic...,992BMXWC,svchost.exe,C:\Windows\System32,4804,NT AUTHORITY\SYSTEM
53,C:\Windows\system32\svchost.exe -k NetworkService,992BMXWC,svchost.exe,C:\Windows\system32,3724,NT AUTHORITY\SYSTEM


name
DllHost.exe      10
WinLogon.exe     10
csrss.exe        13
dllhost.exe      13
lsass.exe        40
services.exe     13
smss.exe         13
spoolsv.exe      40
svchost.exe      40
taskhostw.exe    18
wininit.exe      22
winlogon.exe     30
wmiprvse.exe      9
Name: hostname, dtype: int64
